# sim plan

* for every round
  * procedure?
        * if yes: 
            * offer to random resident (and choose intern who would have gotten it either way for log)
                * if accepts: increment and log to both
                * else: log miss only
        * else
            * log missed procedure0
    * 

# first attempt with no probability:

In [30]:
from collections import namedtuple

import numpy as np
import pandas as pd
from icecream import ic

In [2]:
residents = pd.read_excel("residents.xlsx")
residents

,name,training_level,paracentesis_completed
0,R2 1,senior,0
1,R2 2,senior,0
2,R2 3,senior,0
3,R2 4,senior,0
4,R2 5,senior,0
5,R2 6,senior,0
6,R2 7,senior,0
7,R2 8,senior,0
8,R2 9,senior,0
9,R2 10,senior,0


## note no need to record "signed off" state:

In [3]:
# no need to record "signed off"; can be generated at will
(residents.paracentesis_completed >= 5).rename("signed off")

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10     True
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
Name: signed off, dtype: bool

## anyway:

In [4]:
seniors = residents.training_level == "senior"
residents[seniors]

,name,training_level,paracentesis_completed
0,R2 1,senior,0
1,R2 2,senior,0
2,R2 3,senior,0
3,R2 4,senior,0
4,R2 5,senior,0
5,R2 6,senior,0
6,R2 7,senior,0
7,R2 8,senior,0
8,R2 9,senior,0
9,R2 10,senior,0


In [5]:
interns = residents.training_level == "intern"
residents[interns]

,name,training_level,paracentesis_completed
20,R1 1,intern,0
21,R1 2,intern,1
22,R1 3,intern,0
23,R1 4,intern,0
24,R1 5,intern,0
25,R1 6,intern,0
26,R1 7,intern,0
27,R1 8,intern,0
28,R1 9,intern,0
29,R1 10,intern,0


In [48]:
Procedure_Outcome = namedtuple(
    "Procedure_Outcome",
    [
        "day",
        "senior",
        "senior_prior_procedures",
        "intern",
        "intern_prior_procedures",
        "nth_procedure_opportunity_of_the_day",
        "performed",
    ],
)

In [56]:
days = 2
signoff_threshold = 1
signed_off_avidity = 0.8
not_signed_off_avidity = 0.1

outcome_log = list()


for day in range(days):
    procedure_opportunity_today = 2  # artificial for testing
    for procedure_opportunity in range(procedure_opportunity_today):

        # selected_senior = ic(residents[seniors].sample().name.values[0])
        # selected_intern = ic(residents[interns].sample().name.values[0])

        selected_senior = "R3 2"
        selected_intern = "R1 1"

        if ic(
            (
                residents.loc[
                    residents.name.isin([selected_senior, selected_intern]),
                    "paracentesis_completed",
                ]
                >= signoff_threshold
            ).any()
        ):
            ic("signed off")
            avidity = signed_off_avidity
        else:
            ic("not signed off")
            avidity = not_signed_off_avidity

        ic("residents are:")
        ic(residents.loc[residents.name == selected_senior])
        ic(residents.loc[residents.name == selected_intern])

        performed = None

        if ic(np.random.rand() <= avidity):
            ic("procedure done; new values:")
            senior_prior_procedures = residents.loc[
                residents.name == selected_senior, "paracentesis_completed"
            ].values[0]

            residents.loc[
                residents.name == selected_senior, "paracentesis_completed"
            ] += 1
            
            intern_prior_procedures = residents.loc[
                residents.name == selected_intern, "paracentesis_completed"
            ].values[0]

            residents.loc[
                residents.name == selected_intern, "paracentesis_completed"
            ] += 1
            performed = True
        else:
            ic("no procedure")
            performed = False

        ic("final values:")
        ic(residents.loc[residents.name == selected_senior])
        ic(residents.loc[residents.name == selected_intern])

        today_outcome = ic(
            Procedure_Outcome(
                day=day,
                senior=selected_senior,
                senior_prior_procedures=senior_prior_procedures,
                intern=selected_intern,
                intern_prior_procedures=intern_prior_procedures,
                nth_procedure_opportunity_of_the_day=procedure_opportunity,
                performed=performed,
            )
        )

        outcome_log.append(today_outcome)
        ic(outcome_log)

ic| (
        residents.loc[
            residents.name.isin([selected_senior, selected_intern]),
            "paracentesis_completed",
        ]
        >= signoff_threshold
    ).any(): np.True_
ic| 'signed off'
ic| 'residents are:'
ic| residents.loc[residents.name == selected_senior]:     name training_level  paracentesis_completed
                                                      11  R3 2         senior                       7
ic| residents.loc[residents.name == selected_intern]:     name training_level  paracentesis_completed
                                                      20  R1 1         intern                       5
ic| np.random.rand() <= avidity: True
ic| 'procedure done; new values:'
ic| 'final values:'
ic| residents.loc[residents.name == selected_senior]:     name training_level  paracentesis_completed
                                                      11  R3 2         senior                       8
ic| residents.loc[residents.name == selected_intern]:     nam

In [57]:
pd.DataFrame(outcome_log)

,day,senior,senior_prior_procedures,intern,intern_prior_procedures,nth_procedure_opportunity_of_the_day,performed
0,0,R3 2,7,R1 1,5,0,True
1,0,R3 2,8,R1 1,6,1,True
2,1,R3 2,9,R1 1,7,0,True
3,1,R3 2,10,R1 1,8,1,True
